<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Import the packege

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import shap
from scipy.stats import wilcoxon, bootstrap
import logging
import warnings
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
import random
warnings.filterwarnings('ignore')

## logging fit

In [20]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## seed

In [21]:
SEED = 42  
np.random.seed(SEED)

# read data set

In [22]:
df_xlsx = pd.read_excel('../data/DummyData.xlsx')

df_xlsx.to_csv('../data/DummyData.csv', index=False)
logger.info("change the xlsx form csv file done! ")

2025-06-06 05:59:00,487 - INFO - change the xlsx form csv file done! 


In [23]:
df = pd.read_csv('../data/DummyData.csv')
df.shape
logger.info('read the csv file')

2025-06-06 05:59:00,602 - INFO - read the csv file


In [24]:
df_original = df.copy()
logger.info("copy data set done!")

2025-06-06 05:59:00,744 - INFO - copy data set done!


In [25]:
df.head()

,sex,age,educational_status,partner_status,employment_status,weight,height,BMI,treatment,EMI,...,nitrate,diuretic,psychotropic,angiotensin2_receptor_blocker,week,steps,mvpa_mins,perception,steps_goal,mvpa_mins_goal
0,2,34.0,3.0,0.0,2.0,48.0,164.0,17.85,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,72765,2105,16,95047.0,2509.0
1,2,34.0,3.0,0.0,2.0,48.0,164.0,17.85,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,53817,1557,2,74490.0,2063.0
2,2,34.0,3.0,0.0,2.0,48.0,164.0,17.85,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,58911,1704,4,70379.0,2163.0
3,2,34.0,3.0,0.0,2.0,48.0,164.0,17.85,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,85157,2463,8,125754.0,3164.0
4,2,34.0,3.0,0.0,2.0,48.0,164.0,17.85,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,51341,1485,12,68819.0,1886.0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sex                            566 non-null    int64  
 1   age                            564 non-null    float64
 2   educational_status             561 non-null    float64
 3   partner_status                 561 non-null    float64
 4   employment_status              561 non-null    float64
 5   weight                         561 non-null    float64
 6   height                         561 non-null    float64
 7   BMI                            561 non-null    float64
 8   treatment                      561 non-null    float64
 9   EMI                            559 non-null    float64
 10  EAP                            559 non-null    float64
 11  EPTCA                          559 non-null    float64
 12  ECABG                          559 non-null    flo

In [27]:
df.describe()

,sex,age,educational_status,partner_status,employment_status,weight,height,BMI,treatment,EMI,...,nitrate,diuretic,psychotropic,angiotensin2_receptor_blocker,week,steps,mvpa_mins,perception,steps_goal,mvpa_mins_goal
count,566.000000,564.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,559.000000,...,561.000000,561.000000,561.000000,561.000000,561.000000,566.000000,566.000000,566.000000,531.000000,530.000000
mean,1.526502,48.753546,1.909091,0.520499,1.522282,77.306595,170.602496,26.537683,0.898396,0.157424,...,0.119430,0.160428,0.119430,0.155080,3.522282,42170.001767,1023.332155,5.328622,55226.549906,1260.035849
std,0.499739,14.723789,0.806628,0.500025,0.499949,15.792022,10.074398,4.984085,0.835815,0.364526,...,0.324583,0.367330,0.324583,0.362304,1.925381,22804.693283,624.349263,4.213101,30099.992532,773.527674
min,1.000000,25.000000,1.000000,0.000000,1.000000,46.000000,150.000000,17.170000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,2145.000000,5.000000,0.000000,3089.000000,6.000000
25%,1.000000,36.000000,1.000000,0.000000,1.000000,64.000000,164.000000,22.940000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,23264.000000,604.250000,2.000000,30682.500000,716.250000
50%,2.000000,47.000000,2.000000,1.000000,2.000000,78.000000,170.000000,26.300000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,3.000000,42602.000000,961.000000,5.000000,54642.000000,1153.500000
75%,2.000000,62.000000,3.000000,1.000000,2.000000,90.000000,178.000000,29.970000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,5.000000,58830.250000,1400.750000,8.000000,77647.500000,1743.750000
max,2.000000,75.000000,3.000000,1.000000,2.000000,106.000000,190.000000,38.540000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,8.000000,102990.000000,2745.000000,20.000000,143791.000000,3617.000000


In [28]:
df.columns

Index(['sex', 'age', 'educational_status', 'partner_status',
       'employment_status', 'weight', 'height', 'BMI', 'treatment', 'EMI',
       'EAP', 'EPTCA', 'ECABG', 'ECVA_TIA', 'riskfactor_familyhistory',
       'riskfactor_DM', 'riskfactor_hypertension', 'riskfactor_dyslipidemia',
       'riskfactor_smoking', 'acetylsalicylic_acid',
       'cholesterol_lowering_med', 'ACE_inhibitor', 'calcium_blocker',
       'beta_blocker', 'thienopyridine', 'oral_anticoagulant', 'nitrate',
       'diuretic', 'psychotropic', 'angiotensin2_receptor_blocker', 'week',
       'steps', 'mvpa_mins', 'perception', 'steps_goal', 'mvpa_mins_goal'],
      dtype='object')

In [52]:
# describe the columns
numeric_columns = ['age', 'weight', 'height', 'BMI', 'steps', 'mvpa_mins']
categorical_columns = ['educational_status', 'partner_status', 'employment_status', 
                      'treatment', 'riskfactor_familyhistory', 'riskfactor_DM', 
                      'riskfactor_hypertension', 'riskfactor_dyslipidemia', 
                      'riskfactor_smoking', 'acetylsalicylic_acid', 
                      'cholesterol_lowering_med', 'ACE_inhibitor', 
                      'calcium_blocker', 'beta_blocker', 'thienopyridine', 
                      'nitrate', 'diuretic', 'psychotropic', 
                      'angiotensin2_receptor_blocker']
extra_columns = ['EMI', 'EAP', 'EPTCA', 'ECABG', 'ECVA_TIA']

In [54]:
df = df.dropna(subset=['steps_goal', 'mvpa_mins_goal'])
logger.info("Deleted rows with missing targets")
print(f"THis is the shape of data {df.shape}")

2025-06-06 06:06:39,474 - INFO - Deleted rows with missing targets


THis is the shape of data (566, 36)


In [62]:
df['weight_height_ratio'] = df['weight'] / df['height']
df['steps_week_interaction'] = df['steps'] * df['week']
logger.info("Added new features")

2025-06-06 06:11:29,175 - INFO - Added new features


In [63]:
X = df.drop(['steps_goal', 'mvpa_mins_goal'], axis=1)
y = df[['steps_goal', 'mvpa_mins_goal']]
X_temp, X_holdout, y_temp, y_holdout = train_test_split(X, y, test_size=0.15, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=SEED)
logger.info("Data split into train, validation, and holdout")
print(" X_train:", X_train.shape, " X_val:", X_val.shape, " X_holdout:", X_holdout.shape)

2025-06-06 06:11:30,660 - INFO - Data split into train, validation, and holdout


 X_train: (384, 36)  X_val: (97, 36)  X_holdout: (85, 36)


In [64]:
from sklearn.impute import SimpleImputer

In [65]:
# train 
numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [66]:
X_train[numeric_columns] = numeric_imputer.fit_transform(X_train[numeric_columns])
X_train[categorical_columns + extra_columns] = categorical_imputer.fit_transform(X_train[categorical_columns + extra_columns])

In [67]:
X_val[numeric_columns] = numeric_imputer.transform(X_val[numeric_columns])
X_val[categorical_columns + extra_columns] = categorical_imputer.transform(X_val[categorical_columns + extra_columns])
X_holdout[numeric_columns] = numeric_imputer.transform(X_holdout[numeric_columns])
X_holdout[categorical_columns + extra_columns] = categorical_imputer.transform(X_holdout[categorical_columns + extra_columns])
logger.info("Missing values imputed using train statistics")

2025-06-06 06:11:34,662 - INFO - Missing values imputed using train statistics


In [68]:
for col in numeric_columns + ['weight_height_ratio', 'steps_week_interaction']:
    Q1 = X_train[col].quantile(0.25)
    Q3 = X_train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    X_train[col] = X_train[col].clip(lower=lower_bound, upper=upper_bound)
    X_val[col] = X_val[col].clip(lower=lower_bound, upper=upper_bound)
    X_holdout[col] = X_holdout[col].clip(lower=lower_bound, upper=upper_bound)
logger.info("Outliers clipped using train statistics")

2025-06-06 06:11:35,919 - INFO - Outliers clipped using train statistics


In [69]:
X_train.to_csv('../data/X_train_cleaned.csv', index=False)

## Pipeline

In [73]:
num_features = numeric_columns + categorical_columns + extra_columns + ['weight_height_ratio', 'steps_week_interaction']

In [77]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_regression, k=10))
])
preprocessor = Pipeline([
    ('column_transformer', ColumnTransformer([
        ('num', numeric_transformer, num_features)
    ]))
])

In [78]:
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train['steps_goal'])
X_val_preprocessed = preprocessor.transform(X_val)
X_holdout_preprocessed = preprocessor.transform(X_holdout)

In [80]:
selected_features_mask = preprocessor.named_steps['column_transformer'].named_transformers_['num'].named_steps['selector'].get_support()
selected_features = [num_features[i] for i in range(len(num_features)) if selected_features_mask[i]]
print("featue choose:", selected_features)
logger.info("Feature selection completed")

2025-06-06 06:24:39,078 - INFO - Feature selection completed


featue choose: ['steps', 'treatment', 'riskfactor_hypertension', 'riskfactor_smoking', 'acetylsalicylic_acid', 'beta_blocker', 'angiotensin2_receptor_blocker', 'EAP', 'ECABG', 'steps_week_interaction']


In [81]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from xgboost import XGBRegressor
import pandas as pd

In [89]:
models = {
    'Linear': MultiOutputRegressor(LinearRegression()),
    'Ridge': MultiOutputRegressor(Ridge()),
    'Lasso': MultiOutputRegressor(Lasso()),
    'ElasticNet': MultiOutputRegressor(ElasticNet()),
    'KNN': MultiOutputRegressor(KNeighborsRegressor()),
    'MLP': MultiOutputRegressor(MLPRegressor(max_iter=500, random_state=SEED)),
    'ExtraTrees': MultiOutputRegressor(ExtraTreesRegressor(random_state=SEED)),
    'RandomForest': MultiOutputRegressor(RandomForestRegressor(random_state=SEED)),
    'GradientBoosting': MultiOutputRegressor(GradientBoostingRegressor(random_state=SEED)),

}

In [92]:
estimators = [
    ('rf', RandomForestRegressor(random_state=SEED)),
    ('gb', GradientBoostingRegressor(random_state=SEED))
]
stacked_model = MultiOutputRegressor(StackingRegressor(estimators=estimators, final_estimator=Ridge()))
models['StackedEnsemble'] = stacked_model

In [93]:
results = {}
for name, model in models.items():
    model.fit(X_train_preprocessed, y_train)
    y_pred = model.predict(X_val_preprocessed)
    results[name] = {
        'MAE_steps_goal': mean_absolute_error(y_val['steps_goal'], y_pred[:, 0]),
        'MAE_mvpa_mins_goal': mean_absolute_error(y_val['mvpa_mins_goal'], y_pred[:, 1]),
        'RMSE_steps_goal': np.sqrt(mean_squared_error(y_val['steps_goal'], y_pred[:, 0])),
        'RMSE_mvpa_mins_goal': np.sqrt(mean_squared_error(y_val['mvpa_mins_goal'], y_pred[:, 1])),
        'R2_steps_goal': r2_score(y_val['steps_goal'], y_pred[:, 0]),
        'R2_mvpa_mins_goal': r2_score(y_val['mvpa_mins_goal'], y_pred[:, 1]),
        'MedianAE_steps_goal': median_absolute_error(y_val['steps_goal'], y_pred[:, 0]),
        'MedianAE_mvpa_mins_goal': median_absolute_error(y_val['mvpa_mins_goal'], y_pred[:, 1])
    }
    logger.info(f"Evaluated model: {name}")


results_df = pd.DataFrame(results).T
print("result the model:")
print(results_df)
results_df.to_csv('../data/model_results.csv', index=True)
logger.info("Model results saved to model_results.csv")

2025-06-06 06:37:20,855 - INFO - Evaluated model: Linear
2025-06-06 06:37:20,931 - INFO - Evaluated model: Ridge
2025-06-06 06:37:21,000 - INFO - Evaluated model: Lasso
2025-06-06 06:37:21,118 - INFO - Evaluated model: ElasticNet
2025-06-06 06:37:21,296 - INFO - Evaluated model: KNN


2025-06-06 06:37:34,816 - INFO - Evaluated model: MLP
2025-06-06 06:37:37,981 - INFO - Evaluated model: ExtraTrees
2025-06-06 06:37:41,920 - INFO - Evaluated model: RandomForest
2025-06-06 06:37:43,419 - INFO - Evaluated model: GradientBoosting
2025-06-06 06:38:15,230 - INFO - Evaluated model: StackedEnsemble
2025-06-06 06:38:15,295 - INFO - Model results saved to model_results.csv


result the model:
                  MAE_steps_goal  MAE_mvpa_mins_goal  RMSE_steps_goal  \
Linear               7118.273873          575.567688     10655.250633   
Ridge                7148.214865          575.540793     10664.475001   
Lasso                7118.864042          575.742325     10655.817704   
ElasticNet          11575.124542          589.662158     13981.415434   
KNN                 10380.173196          523.369897     14833.590458   
MLP                 53049.869461         1007.725336     61591.806747   
ExtraTrees           7493.307134          454.294289     12399.025869   
RandomForest         7634.024577          530.555485     12253.201977   
GradientBoosting     7542.349683          583.178553     11714.924771   
StackedEnsemble      7952.140032          552.629764     12260.126447   

                  RMSE_mvpa_mins_goal  R2_steps_goal  R2_mvpa_mins_goal  \
Linear                     754.930112       0.886395           0.111000   
Ridge                      7